In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

RuntimeError: The current Numpy installation ('c:\\users\\sourav kanthal\\anaconda3\\envs\\carprediction\\lib\\site-packages\\numpy\\__init__.py') fails to pass a sanity check due to a bug in the windows runtime. See this issue for more information: https://tinyurl.com/y3dm3h86

In [ ]:
#reading the files
df = pd.read_csv("car data.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

## Data Types

In [ ]:
df.dtypes

## Describe function

In [ ]:
df.describe()

In [ ]:
df.describe(include='all')

## Isnull Function

In [ ]:
#count the number of missing values
df.isnull().sum()

**Their is no null value in the dataset**.

## Unique Function

In [ ]:
df.nunique()

In [ ]:
temp = pd.DataFrame(index = df.columns)
temp['DataType']=df.dtypes
temp['null_count']=df.isnull().sum()
temp['unique_count']=df.nunique()

In [ ]:
temp

In [ ]:
final_dataset = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven','Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

- In the dataset for prediction their is no requirement of car name so we drop it

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['Current_Year']=2020

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['no_year']=final_dataset['Current_Year']-final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Year'],axis=1,inplace=True)
final_dataset.drop(['Current_Year'],axis=1,inplace=True)

- Hence no of year is present so 'Year' and 'Current Year' is useless 

In [ ]:
final_dataset.head()

# Univariate Analysis

## Plotting HIstogram

In [ ]:
final_dataset['Kms_Driven'].plot.hist(bins=50)

In [ ]:
final_dataset.plot.hist(bins=50)

## Using one hot encoding to convert continuous value

In [ ]:
final_dataset = pd.get_dummies(final_dataset)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.columns

In [ ]:
final_dataset.shape

In [ ]:
final_dataset.corr()

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
#plotting correlation between all the features
corrmat = final_dataset.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
sns.heatmap(final_dataset[top_corr_features].corr(),annot=True,cmap='RdYlGn')

## Segregate the Independent and Dependent variable

In [ ]:
x=final_dataset.drop(['Selling_Price'],axis=1)
y=final_dataset['Selling_Price']
x.shape,y.shape

In [ ]:
y.head()

In [ ]:
###Features Importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor(n_estimators=100,min_samples_leaf=1,min_samples_split=2,random_state=56)
model.fit(x,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot the graph on features_importance for better visualization
feature_importance = pd.Series(model.feature_importances_,index=x.columns)
feature_importance.nlargest(5).plot(kind='barh')
plt.show()

## Train_Test_Split

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2,random_state=42)
train_x.shape,train_y.shape,test_x.shape,test_y.shape

In [ ]:
train_x.head()

## Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rr = RandomForestRegressor(random_state=101)

In [ ]:
### Hyperparameters

#number of trees in random forest 
n_estimators = [int(x) for x in np.linspace(start =100,stop=1200,num = 12)]

#number of features to consider at every split
max_features=['auto','sqrt']

#Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5,30,num=6)]

#max_deppth.append(none)

#minimum number of samples required to split a node
min_samples_split = [2,5,10,15,100]

#minimum number of samples required at each leaf node 
min_samples_leaf = [1,2,5,10]


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#create random_grid\
random_grid = {'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf}

In [ ]:
print(random_grid)

In [ ]:
rr_random = RandomizedSearchCV(estimator=rr,param_distributions = random_grid,scoring='neg_mean_squared_error',n_iter = 10, cv = 10,verbose = 2,random_state=56,n_jobs=1)


In [ ]:
rr_random.fit(train_x,train_y)

In [ ]:
predictions = rr_random.predict(test_x)

In [ ]:
predictions

In [ ]:
sns.distplot(test_y-predictions)

In [ ]:
plt.scatter(test_y,predictions)

In [ ]:
import pickle
# open a file wfere I want to store the data
file = open('random_forest_regression_model.pkl','wb')

#dump information to that file
pickle.dump(rr_random,file)